In [1]:
import sys
import os
import argparse
import json

# 添加项目路径
sys.path.append('/Users/haha/Story')  # 你的项目根目录

# 导入所有必要模块
from src.constant import output_dir
from src.utils.utils import save_md, save_json, load_json, extract_plot_list, generate_response, convert_json
from src.generation.outline_generator import generate_outline
from src.generation.chapter_reorder import reorder_chapters
from src.generation.generate_characters import generate_characters_v1
from src.generation.expand_story import expand_story_v1
from src.compile_story import compile_full_story_by_sentence, compile_full_story_by_chapter
from src.enhance_story import enhance_story_with_transitions, polish_dialogues_in_story
from src.generation.dialogue_inserter import analyze_dialogue_insertions, run_dialogue_insertion, analyze_dialogue_insertions_v2
from src.utils.utils import extract_behavior_llm, convert_dialogue_dict_to_list
from src.sync.plot_sync_manager import sync_plot_and_dialogue_from_behavior
from src.sync.auto_propagate_plot_update import auto_propagate_plot_update
from src.analysis.character_state_tracker import run_character_state_tracker
from src.utils.logger import append_log, build_log_record, build_simple_log, init_log_path
from src.version_namer import build_version_name 

print("✅ 所有模块导入成功")

#

✅ 所有模块导入成功


In [2]:
# Cell 1: 导入你的模块
import json
import sys
import os

# 设置项目路径
project_path = "/Users/haha/Story"
sys.path.append(project_path)
test_data_path = "/Users/haha/Story/data/output/test_narrative"

# 直接导入你的工具函数
from src.utils.utils import generate_response, convert_json, load_json, save_json

print("✅ 成功导入项目模块")

✅ 成功导入项目模块


In [3]:
# Cell 2: 加载真实数据
def load_test_data():
    """从你的实际文件中加载测试数据"""
    files_to_check = [
        "test_outline.json",           
        "test_reorder_outline.json",   
        "test_outline_linear.json"     
    ]
    
    loaded_files = {}
    for filename in files_to_check:
        filepath = os.path.join(test_data_path, filename)
        if os.path.exists(filepath):
            try:
                loaded_files[filename] = load_json(filepath)
                print(f"✅ 加载成功: {filename}")
            except Exception as e:
                print(f"❌ 加载失败: {filename} - {e}")
        else:
            print(f"📁 文件不存在: {filename}")
    
    return loaded_files

data_files = load_test_data()

# 提取章节数据
def extract_chapter_data(data_files):
    original_chapters = None
    if "test_outline.json" in data_files:
        original_chapters = data_files["test_outline.json"]
    elif "test_outline_linear.json" in data_files:
        original_chapters = data_files["test_outline_linear.json"]
    
    reordered_chapters = None
    if "test_reorder_outline.json" in data_files:
        reordered_chapters = data_files["test_reorder_outline.json"]
    
    return original_chapters, reordered_chapters

original_chapters, reordered_chapters = extract_chapter_data(data_files)

print("🔍 数据检查:")
if original_chapters:
    print(f"  原始章节: {len(original_chapters)} 个")
else:
    print("  ⚠️ 未找到原始章节数据，使用备用数据")
    # 使用备用数据...

📁 文件不存在: test_outline.json
✅ 加载成功: test_reorder_outline.json
✅ 加载成功: test_outline_linear.json
🔍 数据检查:
  原始章节: 7 个


In [4]:
# Cell 3: 修正版的改进函数
def analyze_narrative_structure_improved(reordered_chapters, original_chapters, topic="未知", style="未知"):
    """修正版 - 使用你的工具函数"""
    
    chapter_analysis = []
    for new_pos, ch in enumerate(reordered_chapters):
        original_pos = next((i for i, orig_ch in enumerate(original_chapters) 
                           if orig_ch["chapter_id"] == ch["chapter_id"]), -1)
        
        # 计算位置变化
        position_change = new_pos - original_pos if original_pos >= 0 else 0
        
        # 时间关系分析
        if position_change > 0:
            time_relation = "时间前进"
            narrative_hint = "顺序发展，可直接承接"
        elif position_change < 0:
            time_relation = "时间倒退" 
            narrative_hint = "需要倒叙/插叙处理"
        else:
            time_relation = "位置不变"
            narrative_hint = "保持原有节奏"
        
        # 角色关系上下文分析
        if new_pos == 0:
            character_context = "读者首次接触，需要建立角色认知"
        elif original_pos == 0 and new_pos > 0:
            character_context = "原开头章节，现需要补充背景"
        elif abs(position_change) > 2:
            character_context = "大幅位置变动，需要仔细处理角色关系"
        else:
            character_context = "正常承接，维持角色关系连贯性"
            
        chapter_analysis.append({
            "章节ID": ch["chapter_id"],
            "标题": ch["title"], 
            "摘要": next((orig["summary"] for orig in original_chapters 
                         if orig["chapter_id"] == ch["chapter_id"]), ""),
            "新位置": new_pos + 1,
            "原位置": original_pos + 1 if original_pos >= 0 else "未知",
            "位置变化": position_change,
            "时间关系": time_relation,
            "叙述提示": narrative_hint,
            "角色关系上下文": character_context,
            "位置跨度": abs(position_change)
        })
    
    # ✅ 修正：使用你的 json.dumps
    info_text = json.dumps(chapter_analysis, ensure_ascii=False, indent=2)
    
    # ✅ 修正：简化 prompt 格式，避免三重反引号冲突
    prompt = f"""你是专业的叙事结构专家。请为以下非线性小说章节安排设计精确的叙述策略。

【故事背景】
题材：{topic}
风格：{style}

【详细章节时间线分析】
{info_text}

【关键要求】
1. 叙述视角统一：全文必须使用同一种叙述视角，不得随意切换
2. 时间线逻辑：对于"时间倒退"的章节，必须明确使用倒叙或插叙技巧
3. 角色关系一致性：确保角色间的熟悉程度符合故事内在时间线
4. 过渡自然性：章节间的转换要流畅，避免突兀

【特殊注意事项】
- 当"位置变化"为负数时，这是时间倒退，需要特殊处理
- 当"角色关系上下文"显示首次接触时，角色不能表现得已经熟悉
- 避免在不同章节使用不同的叙述人称（第一人称/第三人称混用）

【输出格式】
返回JSON数组，每个章节包含以下字段：
- chapter_id: 章节编号
- narrative_role: 叙述角色（3-6字）
- narrative_instruction: 详细指导（必须包含具体的视角类型、时态处理、角色关系处理方式）
- transition_hint: 具体过渡方法（如何从上一章自然过渡到本章）
- timeline_method: 时间线处理（顺叙/倒叙/插叙/并行叙述）
- pov_requirement: 视角要求（必须统一：第三人称全知/第三人称限知/第一人称）
- character_consistency_note: 角色关系注意事项

请确保所有章节的pov_requirement字段完全一致。只返回JSON数组，不要其他内容。"""
    
    try:
        # ✅ 使用你的函数
        response = generate_response([{"role": "user", "content": prompt}])
        result = convert_json(response)
        
        # 验证结果
        if not isinstance(result, list):
            print("⚠️ LLM返回格式不是数组，尝试修复...")
            return add_basic_analysis(reordered_chapters, original_chapters)
        
        # 检查章节完整性
        result_ids = [item.get("chapter_id") for item in result]
        expected_ids = [ch["chapter_id"] for ch in reordered_chapters]
        
        if set(result_ids) != set(expected_ids):
            print("⚠️ 章节不完整，使用基础分析...")
            return add_basic_analysis(reordered_chapters, original_chapters)
        
        print("✅ LLM分析成功")
        return result
        
    except Exception as e:
        print(f"⚠️ API调用失败: {e}")
        return add_basic_analysis(reordered_chapters, original_chapters)

def add_basic_analysis(reordered_chapters, original_chapters):
    """基础分析（兜底方案）"""
    analyzed_chapters = []
    
    for idx, ch in enumerate(reordered_chapters):
        original_pos = next((i for i, orig_ch in enumerate(original_chapters) 
                           if orig_ch["chapter_id"] == ch["chapter_id"]), -1)
        
        # 简单逻辑判断
        if idx == 0:
            if original_pos > len(original_chapters) // 2:
                role = "悬念开篇"
                instruction = "第三人称限知视角，以后期情节开场营造悬念，不解释前因后果"
                timeline_method = "跳跃开场"
            else:
                role = "正常开篇"
                instruction = "第三人称限知视角，按原有逻辑开场，为后续发展做铺垫"
                timeline_method = "顺叙"
        elif original_pos < idx:
            role = "回忆讲述"
            instruction = "第三人称限知视角，使用回忆形式讲述，添加'让我们回到...'等过渡语言"
            timeline_method = "倒叙"
        else:
            role = "顺序发展"
            instruction = "第三人称限知视角，承接前文，按时间顺序推进故事"
            timeline_method = "顺叙"
        
        analyzed_ch = {
            "chapter_id": ch["chapter_id"],
            "narrative_role": role,
            "narrative_instruction": instruction,
            "transition_hint": "标准过渡",
            "timeline_method": timeline_method,
            "pov_requirement": "第三人称限知",  # ✅ 强制统一视角
            "character_consistency_note": "保持角色关系逻辑"
        }
        
        analyzed_chapters.append(analyzed_ch)
    
    return analyzed_chapters

In [5]:
# Cell 4: 运行测试
print("🧪 开始测试改进版分析函数...")

if original_chapters and reordered_chapters:
    result = analyze_narrative_structure_improved(
        reordered_chapters, original_chapters,
        topic="小红帽", style="ABO改写"
    )
    
    print(f"✅ 分析完成，返回 {len(result)} 个章节")
    
    # 检查视角一致性
    povs = [ch.get('pov_requirement') for ch in result]
    unique_povs = set(povs)
    
    print(f"\n👁 视角一致性检查:")
    print(f"  使用的视角: {unique_povs}")
    if len(unique_povs) == 1:
        print("  ✅ 视角统一！")
    else:
        print("  ❌ 视角不统一，需要修正")
    
    # 显示结果
    print(f"\n📖 分析结果预览:")
    for ch in result[:3]:  # 只显示前3个
        print(f"  {ch['chapter_id']}: {ch['narrative_role']}")
        print(f"    视角: {ch['pov_requirement']}")
        print(f"    时间线: {ch['timeline_method']}")
        print(f"    指导: {ch['narrative_instruction'][:80]}...")
        print()
    
    # 保存结果
    result_file = os.path.join(test_data_path, "improved_narrative_analysis.json")
    save_json(result, "test_narrative", "improved_narrative_analysis.json")
    print(f"📁 结果已保存: improved_narrative_analysis.json")
    
else:
    print("❌ 缺少测试数据")

🧪 开始测试改进版分析函数...
原始 content: [
  {
    "chapter_id": "Chapter 5",
    "narrative_role": "场景奠基",
    "narrative_instruction": "使用第三人称限知视角，以小红帽为主视点，采用过去时，开篇设置紧张氛围，强调小红帽初到外婆家对周围环境的陌生感与警觉心，角色间表现首次交流，避免任何提前熟悉。引入狼的信息素诱惑，通过小红帽的感受展现异样，但不提前泄露狼真实身份。",
    "transition_hint": "以小红帽抵达门口、推开门的一瞬作为章节开端，将读者直接带入屋内场景，营造悬念。",
    "timeline_method": "倒叙",
    "pov_requirement": "第三人称限知",
    "character_consistency_note": "角色首次接触，需保持陌生和试探，避免表现出已有信任或了解。"
  },
  {
    "chapter_id": "Chapter 2",
    "narrative_role": "危险预示",
    "narrative_instruction": "继续使用第三人称限知视角，主视点仍为小红帽，采用过去时，承接上一章紧张氛围，展现她在森林中与Alpha猎人的偶遇，猎人与她之间为初次交流，角色关系生疏，猎人的警告被小红帽质疑，突出她的独立与警觉。",
    "transition_hint": "通过小红帽回忆刚才在外婆家感到的异样，引出她路上曾遇猎人并被警告的往事，形成自然插叙。",
    "timeline_method": "插叙",
    "pov_requirement": "第三人称限知",
    "character_consistency_note": "小红帽与猎人初次互动，保持陌生和防备，避免后期信任感提前显露。"
  },
  {
    "chapter_id": "Chapter 4",
    "narrative_role": "阴谋铺垫",
    "narrative_instruction": "采用第三人称限知视角，主视点切回小红帽，但通过她的观察和推理，间接展现狼在外婆家实施阴谋的过程。使用过去时，情节紧凑，让读者随

In [6]:
# Cell 5: 检查是否能替换现有代码
def validate_result_format(result):
    """验证结果格式是否符合你的代码要求"""
    required_fields = [
        "chapter_id", "narrative_role", "narrative_instruction", 
        "transition_hint", "timeline_method", "pov_requirement"
    ]
    
    print("🔍 格式验证:")
    for i, ch in enumerate(result):
        missing_fields = [field for field in required_fields if field not in ch]
        if missing_fields:
            print(f"  章节 {i+1} 缺少字段: {missing_fields}")
        else:
            print(f"  章节 {i+1} 格式完整 ✅")
    
    # 检查字段值是否合理
    print(f"\n📊 字段统计:")
    print(f"  视角类型: {set(ch.get('pov_requirement') for ch in result)}")
    print(f"  时间线方法: {set(ch.get('timeline_method') for ch in result)}")
    
    return len([ch for ch in result if all(field in ch for field in required_fields)]) == len(result)

if 'result' in locals() and result:
    is_valid = validate_result_format(result)
    print(f"\n🎯 格式验证结果: {'通过' if is_valid else '不通过'}")
    
    if is_valid:
        print("✅ 可以替换到你的 narrative_analyzer.py 中！")
    else:
        print("⚠️ 需要进一步调整格式")

🔍 格式验证:
  章节 1 格式完整 ✅
  章节 2 格式完整 ✅
  章节 3 格式完整 ✅
  章节 4 格式完整 ✅
  章节 5 格式完整 ✅
  章节 6 格式完整 ✅
  章节 7 格式完整 ✅

📊 字段统计:
  视角类型: {'第三人称限知'}
  时间线方法: {'插叙', '顺叙', '倒叙'}

🎯 格式验证结果: 通过
✅ 可以替换到你的 narrative_analyzer.py 中！
